<a href="https://colab.research.google.com/github/JahnaviKumar8/voice-based-emotion-recognition-using-Transformers/blob/main/n1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub
!apt-get install -y ffmpeg

!pip install accelerate -U

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
ERROR: unknown command "install accelerate -U"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!pip install datasets
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
import random
from collections import Counter
from sklearn.metrics import classification_report, accuracy_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
import random
from collections import Counter
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
def load_audio_files(audio_path, target_sr=16000, max_duration=35):
    audio_data = []
    labels = []

    e_f = os.listdir(audio_path)

    for emotion in  e_f:
        e_p = os.path.join(audio_path, emotion)
        audio_files = os.listdir(e_p)

        for audio_file in audio_files[:10]:  # Limit to 10 files per singer for now
            file_path = os.path.join(e_p, audio_file)

            # Load and resample audio
            audio, sr = librosa.load(file_path, sr=target_sr, duration=max_duration)

            # Normalize
            audio = audio.astype(np.float32) / np.max(np.abs(audio))

            audio_data.append(audio)
            labels.append(emotion)

    return np.array(audio_data, dtype=object), np.array(labels)


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
audio_path = '/content/drive/MyDrive/ColabDrive/total'
audio_data, labels = load_audio_files(audio_path)

unique_labels = np.unique(labels)
print("labels (emotion names):", unique_labels)

<ipython-input-5-bf9155bc7a30>:15: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=target_sr, duration=max_duration)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-5-bf9155bc7a30>:15: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=target_sr, duration=max_duration)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-5-bf9155bc7a30>:15: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(fil

labels (emotion names): ['clips_angry' 'clips_happy' 'clips_sad']


In [ ]:
label_to_id = {label: i for i, label in enumerate(unique_labels)}
id_to_label = {i: label for label, i in label_to_id.items()}

In [ ]:
label_ids = np.array([label_to_id[label] for label in labels])

# Split the data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(
    audio_data, label_ids, test_size=0.2, random_state=42
)

# Create datasets
train_dataset = Dataset.from_dict({"input_values": train_data.tolist(), "label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"input_values": val_data.tolist(), "label": val_labels.tolist()})

# Initialize feature extractor and model for audio classification
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")
classification_model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base",
    num_labels=len(unique_labels),
    ignore_mismatched_sizes=True
)

# Define the data collator
def data_collator(features):
    input_features = [feature["input_values"] for feature in features]
    label_ids = [feature["label"] for feature in features]

    inputs = feature_extractor(
        input_features,
        sampling_rate=16000,
        padding=True,
        return_tensors="pt"
    )

    inputs["labels"] = torch.tensor(label_ids)
    return inputs



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=100,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    fp16=True,  # Enable mixed precision training for GPU
    save_steps=1000,
    eval_steps=1000,
    dataloader_num_workers=2,
    max_grad_norm=0.5,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=classification_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=feature_extractor,
    data_collator=data_collator,
)


In [ ]:
# Define the data collator
def data_collator(features):
    input_features = [feature["input_values"] for feature in features]
    label_ids = [feature["label"] for feature in features]

    inputs = feature_extractor(
        input_features,
        sampling_rate=16000,
        padding=True,
        return_tensors="pt"
    )

    # Ensure labels are converted to float32
    inputs["labels"] = torch.tensor(label_ids, dtype=torch.float32)
    return inputs

In [ ]:
trainer.train()

def test_multiple_samples(model, feature_extractor, audio_path, id_to_label, num_samples=30):
    predictions = []
    true_labels = []

    for emotion in os.listdir(audio_path):
        e_p = os.path.join(audio_path, emotion)
        audio_files = os.listdir(e_p)

        for _ in range(num_samples // len(os.listdir(audio_path))):
            audio_file = random.choice(audio_files)
            file_path = os.path.join(e_p, audio_file)

            audio, _ = librosa.load(file_path, sr=16000, duration=35)
            inputs = feature_extractor(audio, sampling_rate=16000, return_tensors="pt")
            inputs = {k: v.to('cuda').half() for k, v in inputs.items()}

            with torch.no_grad():
                logits = model(**inputs).logits
            predicted_class_id = logits.argmax().item()
            predicted_label = id_to_label[predicted_class_id]

            predictions.append(predicted_label)
            true_labels.append(emotion)

    accuracy = accuracy_score(true_labels, predictions)
    return predictions, true_labels, accuracy


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch,Training Loss,Validation Loss
0,No log,1.102214
2,No log,1.102214
4,No log,1.102051
6,No log,1.101725
8,No log,1.100260
10,No log,1.098958
12,No log,1.095378
14,No log,1.090658
16,No log,1.084147
18,No log,1.077474


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
import torch
from collections import Counter
from sklearn.metrics import classification_report
import librosa
import random
import os
from sklearn.metrics import accuracy_score

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Save the trained model
trainer.save_model("./trained_audio_classifier")

# Move model to the appropriate device and convert to half precision if using GPU
classification_model = classification_model.to(device)
if device.type == 'cuda':
    classification_model = classification_model.half()

# Function to run multi-sample test
def test_multiple_samples(model, feature_extractor, audio_path, id_to_label, num_samples=30):
    predictions = []
    true_labels = []

    for emotion in os.listdir(audio_path):
        emotion_path = os.path.join(audio_path, emotion)
        audio_files = os.listdir(emotion_path)

        for _ in range(num_samples // len(os.listdir(audio_path))):
            audio_file = random.choice(audio_files)
            file_path = os.path.join(emotion_path, audio_file)

            audio, _ = librosa.load(file_path, sr=16000, duration=5)
            inputs = feature_extractor(audio, sampling_rate=16000, return_tensors="pt")
            inputs = {k: v.to(device).half() if device.type == 'cuda' else v.to(device) for k, v in inputs.items()}

            with torch.no_grad():
                logits = model(**inputs).logits
            predicted_class_id = logits.argmax().item()
            predicted_label = id_to_label[predicted_class_id]

            predictions.append(predicted_label)
            true_labels.append(emotion)

    accuracy = accuracy_score(true_labels, predictions)
    return predictions, true_labels, accuracy # Added return statement


# Run multi-sample test
predictions, true_labels, accuracy = test_multiple_samples(classification_model, feature_extractor, audio_path, id_to_label)

# Print results
print("Prediction distribution:", Counter(predictions))
print("True label distribution:", Counter(true_labels))
print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(true_labels, predictions))

Using device: cuda


<ipython-input-14-1d0eb698224a>:34: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file_path, sr=16000, duration=5)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-14-1d0eb698224a>:34: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file_path, sr=16000, duration=5)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-14-1d0eb698224a>:34: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file_path, sr=16000, duration=5)


Prediction distribution: Counter({'clips_sad': 13, 'clips_happy': 10, 'clips_angry': 7})
True label distribution: Counter({'clips_angry': 10, 'clips_happy': 10, 'clips_sad': 10})
Accuracy: 0.8333333333333334

Classification Report:
              precision    recall  f1-score   support

 clips_angry       0.86      0.60      0.71        10
 clips_happy       1.00      1.00      1.00        10
   clips_sad       0.69      0.90      0.78        10

    accuracy                           0.83        30
   macro avg       0.85      0.83      0.83        30
weighted avg       0.85      0.83      0.83        30



In [ ]:
import torch
if torch.cuda.is_available():
    print("CUDA is available. You can use your GPU.")
else:
    print("CUDA is not available. Using CPU instead.")

CUDA is available. You can use your GPU.


In [ ]:
test_audio, _ = librosa.load(os.path.join(audio_path, unique_labels[0], os.listdir(os.path.join(audio_path, unique_labels[0]))[0]), sr=16000, duration=5)
test_input = feature_extractor(test_audio, sampling_rate=16000, return_tensors="pt")
test_input = {k: v.to('cuda').half() for k, v in test_input.items()}

<ipython-input-16-8f59b7457536>:1: UserWarning: PySoundFile failed. Trying audioread instead.
  test_audio, _ = librosa.load(os.path.join(audio_path, unique_labels[0], os.listdir(os.path.join(audio_path, unique_labels[0]))[0]), sr=16000, duration=5)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
import json
import os
# Save the label mapping
label_map_path = os.path.join("./trained_audio_classifier", "label_map.json")
with open(label_map_path, "w") as f:
    json.dump(id_to_label, f)

In [ ]:
with torch.no_grad():
    logits = classification_model(**test_input).logits
predicted_class_id = logits.argmax().item()
predicted_label = id_to_label[predicted_class_id]
print(f"\nPredicted emotion for single sample: {predicted_label}")


Predicted emotion for single sample: clips_angry


In [ ]:
import os
import torch
import torchaudio
import numpy as np

# Test the model on a single sample
def test_single_sample(model, feature_extractor, audio_path, id_to_label, unique_labels):
    # Randomly select a singer and a song
    test_singer = random.choice(unique_labels)
    test_singer_path = os.path.join(audio_path, test_singer)
    test_audio_file = random.choice(os.listdir(test_singer_path))
    test_audio_path = os.path.join(test_singer_path, test_audio_file)

    print(f"\nTesting on a single sample:")
    print(f"Selected singer: {test_singer}")
    print(f"Selected audio file: {test_audio_file}")

    # Load and preprocess the audio
    waveform, sample_rate = torchaudio.load(test_audio_path)
    if sample_rate != 16000:
        waveform = torchaudio.functional.resample(waveform, sample_rate, 16000)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Trim or pad to 20 seconds
    if waveform.shape[1] > 20 * 16000:
        waveform = waveform[:, :20 * 16000]
    else:
        waveform = torch.nn.functional.pad(waveform, (0, 20 * 16000 - waveform.shape[1]))

    waveform = waveform.squeeze().numpy()

    # Prepare input for the model
    test_input = feature_extractor(waveform, sampling_rate=16000, return_tensors="pt", padding="max_length", max_length=int(16000 * 20), truncation=True)
    test_input = {k: v.to('cuda').half() for k, v in test_input.items()}  # Move input to GPU and convert to half precision

    # Get model prediction
    with torch.no_grad():
        output = model(**test_input)
        logits = output.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)

    # Get predicted class and probability
    predicted_class_id = logits.argmax().item()
    predicted_label = id_to_label[predicted_class_id]
    predicted_probability = probabilities[0][predicted_class_id].item()

    print(f"\nPredicted singer: {predicted_label}")
    print(f"Confidence: {predicted_probability:.2%}")
    print(f"Actual singer: {test_singer}")
    print("\nTop 3 predictions:")

    # Get top 3 predictions
    top3_prob, top3_indices = torch.topk(probabilities, 3)
    for i in range(3):
        print(f"{id_to_label[top3_indices[0][i].item()]}: {top3_prob[0][i].item():.2%}")

    return predicted_label, test_singer

# Add this to your main code after training and before the multi-sample test
print("\n--- Single Sample Test ---")
predicted_label, actual_label = test_single_sample(classification_model, feature_extractor, audio_path, id_to_label, unique_labels)


--- Single Sample Test ---

Testing on a single sample:
Selected singer: clips_angry
Selected audio file: New Project - Made with Clipchamp (3).mp4

Predicted singer: clips_angry
Confidence: 65.42%
Actual singer: clips_angry

Top 3 predictions:
clips_angry: 65.42%
clips_sad: 21.68%
clips_happy: 12.89%


In [ ]:
 import json

# Save the label mapping
label_map_path = os.path.join("./trained_audio_classifier", "label_map.json")
with open(label_map_path, "w") as f:
    json.dump(id_to_label, f)

In [ ]:
import torch
import torchaudio
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
import json
import os

# Load your trained model and feature extractor
model_name = "./trained_audio_classifier"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModelForAudioClassification.from_pretrained(model_name)

# Load the label mapping
label_map_path = os.path.join(model_name, "label_map.json")
with open(label_map_path, "r") as f:
    id_to_label = json.load(f)

# Function to preprocess and predict
def predict_singer(audio_file_path):
    # Load the audio file
    waveform, sample_rate = torchaudio.load(audio_file_path)

    # Resample if necessary (assuming your model expects 16kHz)
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(sample_rate, 16000)
        waveform = resampler(waveform)

    # Convert to mono if stereo
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Preprocess the audio
    inputs = feature_extractor(waveform.numpy()[0], sampling_rate=16000, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits

    # Get the predicted class
    predicted_class_id = logits.argmax().item()
    predicted_singer = id_to_label[str(predicted_class_id)]

    return predicted_singer

# Directory containing the input MP3 files
input_directory = "/content/drive/MyDrive/INPUT"

# Get all MP3 files in the input directory
mp3_files = [f for f in os.listdir(input_directory) if f.endswith('.mp3')]

# Process each MP3 file
for mp3_file in mp3_files:
    audio_file_path = os.path.join(input_directory, mp3_file)
    try:
        predicted_singer = predict_singer(audio_file_path)
        print(f"The predicted singer for {mp3_file} is: {predicted_singer}")
    except Exception as e:
        print(f"Error processing {mp3_file}: {str(e)}")

print("Classification complete.")


Classification complete.


In [ ]:
import torch
import torchaudio
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
import json
import os

# Load your trained model and feature extractor
model_name = "./trained_audio_classifier"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModelForAudioClassification.from_pretrained(model_name)

# Load the label mapping
label_map_path = os.path.join(model_name, "label_map.json")
with open(label_map_path, "r") as f:
    id_to_label = json.load(f)

# Function to preprocess and predict
def predict_singer(audio_file_path):
    # Load the audio file
    waveform, sample_rate = torchaudio.load(audio_file_path)

    # Resample if necessary (assuming your model expects 16kHz)
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(sample_rate, 16000)
        waveform = resampler(waveform)

    # Convert to mono if stereo
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Preprocess the audio
    inputs = feature_extractor(waveform.numpy()[0], sampling_rate=16000, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits

    # Get the predicted class
    predicted_class_id = logits.argmax().item()
    predicted_singer = id_to_label[str(predicted_class_id)]

    return predicted_singer

# Directory containing the input MP3 files
input_directory = "/content/drive/MyDrive/INPUT"

# Get all MP3 files in the input directory
mp3_files = [f for f in os.listdir(input_directory) if f.endswith('.mp3')]

# Ensure we have exactly 9 MP3 files
if len(mp3_files) != 9:
    print(f"Warning: Expected 9 MP3 files, but found {len(mp3_files)} files.")

# Process each MP3 file
for mp3_file in mp3_files:
    audio_file_path = os.path.join(input_directory, mp3_file)
    try:
        predicted_singer = predict_singer(audio_file_path)
        print(f"The predicted singer for {mp3_file} is: {predicted_singer}")
    except Exception as e:
        print(f"Error processing {mp3_file}: {str(e)}")

print("Classification complete.")


Classification complete.


In [ ]:
import torch
import torchaudio
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

# Load your trained model and feature extractor
model_name = "./trained_audio_classifier"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModelForAudioClassification.from_pretrained(model_name)

# Function to preprocess and predict
def predict_singer(audio_file_path):
    # Load the audio file
    waveform, sample_rate = torchaudio.load(audio_file_path)

    # Resample if necessary (assuming your model expects 16kHz)
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(sample_rate, 16000)
        waveform = resampler(waveform)

    # Convert to mono if stereo
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Preprocess the audio
    inputs = feature_extractor(waveform.numpy()[0], sampling_rate=16000, return_tensors="pt")

    # Make prediction
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
# Map the class ID to singer name (you'll need to create this mapping)
    singer_names = {0:"SP BALASUBRAMANYUM", 1:"SHANKAR MAHADEVAN", 2:"SHREYA GHOSHAL"}  # Replace with your actual singer names
    predicted_singer = singer_names[predicted_class_id]

    return predicted_singer

# Use the function
audio_file_path = "/content/drive/MyDrive/INPUT/sg3.mp3"
predicted_singer = predict_singer(audio_file_path)
print(f"The predicted singer is: {predicted_singer}")


RuntimeError: Failed to open the input "/content/drive/MyDrive/INPUT/sg3.mp3" (No such file or directory).
Exception raised from get_input_format_context at /__w/audio/audio/pytorch/audio/src/libtorio/ffmpeg/stream_reader/stream_reader.cpp:42 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7d0fcc578897 in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x64 (0x7d0fcc528b25 in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x42334 (0x7d0ff118d334 in /usr/local/lib/python3.10/dist-packages/torio/lib/libtorio_ffmpeg4.so)
frame #3: torio::io::StreamingMediaDecoder::StreamingMediaDecoder(std::string const&, std::optional<std::string> const&, std::optional<std::map<std::string, std::string, std::less<std::string>, std::allocator<std::pair<std::string const, std::string> > > > const&) + 0

In [ ]:
import torch
import torchaudio
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
import os
import torch.nn.functional as F

# Load the trained model and feature extractor
model_name = "./trained_audio_classifier"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModelForAudioClassification.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def predict_singer(audio_file_path, confidence_threshold=0.5):
    try:
        # Load the audio file
        waveform, sample_rate = torchaudio.load(audio_file_path)

        # Resample if necessary (assuming 16kHz is required)
        if sample_rate != 16000:
            resampler = torchaudio.transforms.Resample(sample_rate, 16000)
            waveform = resampler(waveform)

        # Convert to mono if stereo
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)

        # Normalize
        waveform = waveform / torch.max(torch.abs(waveform))

        # Preprocess the audio
        inputs = feature_extractor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt", padding=True)
        {k: v.to(device) for k, v in inputs.items()}

        # Make prediction
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

        # Apply softmax to get probabilities
        probs = F.softmax(logits, dim=1)

        # Get the predicted class and its probability
        predicted_class_id = logits.argmax().item()
        predicted_prob = probs[0][predicted_class_id].item()

        # Map the class ID to singer name
        id_to_label = {0: "SP BALASUBRAMANYUM", 1: "SHANKAR MAHADEVAN", 2: "SHREYA GHOSHAL"}
        predicted_singer = id_to_label[predicted_class_id]

        # Print probabilities for all classes
        print(f"Probabilities for {os.path.basename(audio_file_path)}:")
        for i, prob in enumerate(probs[0]):
            print(f"{id_to_label[i]}: {prob.item():.4f}")

        # Check if the prediction meets the confidence threshold
        if predicted_prob >= confidence_threshold:
            return predicted_singer, predicted_prob
        else:
            return "Uncertain", predicted_prob

    except Exception as e:
        print(f"Error processing {audio_file_path}: {str(e)}")
        return None, None

# Process all audio files in the input directory
input_directory = "/content/drive/MyDrive/INPUT"

# List to store valid audio files
valid_audio_files = []

# Collect valid audio files
for filename in os.listdir(input_directory):
    if filename.endswith((".mp3", ".wav", ".ogg")):  # Add or remove file extensions as needed
        valid_audio_files.append(filename)

# Ensure we process only 9 files
num_files_to_process = min(9, len(valid_audio_files))

print(f"Processing {num_files_to_process} audio files:")

for i in range(num_files_to_process):
    filename = valid_audio_files[i]
    audio_file_path = os.path.join(input_directory, filename)
    predicted_singer, confidence = predict_singer(audio_file_path)
    if predicted_singer:
        print(f"The predicted singer for {filename} is: {predicted_singer} (Confidence: {confidence:.4f})")
    print()  # Add a blank line for readability

print("\nPrediction completed for all audio files.")

Processing 0 audio files:

Prediction completed for all audio files.
